In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import datetime as dt

import pandas as pd

from deep_hedging import Ticker, Underlyings, NelsonSiegelCurve
from deep_hedging.non_linear.exotic.basket import WorstOfDigitalCall
from deep_hedging.non_linear.exotic.quanto import QuantoOption

RANDOM_SEED = 12
PATH = Path("../../data/")

In [3]:
N_YEARS_LOOKBACK = 5
N_PATHS = 100_000

FIXING = dt.datetime(2021, 4, 19)
FINAL_FIXING = dt.datetime(2024, 4, 17)

In [4]:
underlyings = Underlyings(
    tickers=[
        Ticker("Sanofi", "SAN.PA"),
        Ticker("GlaxoSmithKline PLC", "GSK.L"),
        Ticker("Bayer AG", "BAYN.DE"),
    ],
    start=N_YEARS_LOOKBACK,
    end=FIXING,
)

[*********************100%%**********************]  3 of 3 completed


In [5]:
fx = Underlyings(
    tickers=[
        Ticker("EURRUB", "EURRUB=X"),
        Ticker("GBPRUB", "GBPRUB=X"),
        Ticker("EURRUB", "EURRUB=X"),
    ],
    start=N_YEARS_LOOKBACK,
    end=FIXING,
)

# Issue with yahoo finance data - strange GBPRUB quote
fx.data = fx.data[fx["GBPRUB=X"] > 1.2]
fx.data

[*********************100%%**********************]  2 of 2 completed


Ticker,EURRUB=X,GBPRUB=X,EURRUB=X_2
Date,,,
2004-02-13,36.470001,53.591999,36.470001
2004-04-01,34.990002,52.884998,34.990002
2004-07-27,35.070000,52.995998,35.070000
2005-01-17,36.619999,52.074001,36.619999
2005-02-22,36.689999,53.369999,36.689999
...,...,...,...
2021-04-12,92.128601,106.120071,92.128601
2021-04-13,92.092499,106.289673,92.092499
2021-04-14,90.795601,104.455833,90.795601


In [6]:
nss_params = pd.read_csv(PATH / "rub_nss.csv", sep=";")
rub_params = nss_params[
    nss_params["tradedate"] == f"{FIXING.strftime('%d.%m.%Y')}"
].loc[:, ["B1", "B2", "B3", "T1"]]
rub_params = rub_params.replace(",", ".", regex=True).astype(float)
rub_params

,B1,B2,B3,T1
1833,744.953996,-273.365215,-271.811557,0.905105


In [7]:
curve = NelsonSiegelCurve(
    b0=rub_params.B1.values[0] / 100,
    b1=rub_params.B2.values[0] / 100,
    b2=rub_params.B3.values[0] / 100,
    tau=rub_params.T1.values[0] / 100,
)
curve.get_rate(1.0)

0.07400815995797354

In [8]:
worst_of_digital = WorstOfDigitalCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=1.0,
    frequency=0.5,
    start_date=FIXING,
    end_date=FINAL_FIXING,
    digital_coupon=0.05,
    random_seed=RANDOM_SEED,
)
price = worst_of_digital.price(spot=[1.0] * len(underlyings))
print(f"{100 * price:,.4f}")

4.3762


In [9]:
quanto = QuantoOption(
    option=worst_of_digital,
    modifying_underlyings=fx,
    yield_curve=curve,
    random_seed=RANDOM_SEED,
)
price = quanto.price(spot=[1.0] * (2 * len(underlyings)))
print(f"{100 * price:,.4f} RUB")

8.3050 RUB
